# SVM on the Gene Expression

In [1]:
library(e1071)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
gdsc = readRDS('data/gdsc_screened.rds')
gene = readRDS('data/gene_expression.rds')

In [3]:
gene_label = colnames(gene)# all the genes

Doing feature selection for SVM is challenging. I have 2 alternatives: one is doing T test and pick top features with most extreme T statistics. The second way is doing PCA and pick the top PCs.

In [4]:
codes <- c("1006","1007","1008","1011","1014","1016","1019","1026","1032","1054","1060","1062")
n_features = c(2,3,5,10,50,100,200)
resultmat = matrix(nrow=12,ncol=7)
rownames(resultmat) = c("1006","1007","1008","1011","1014","1016","1019","1026","1032","1054","1060","1062")
colnames(resultmat) = c("2","3","5","10","50","100","200")

In [11]:
for(code in codes){
    gdsc_sub <- subset(gdsc, DRUG_ID_lib == code)[,c("CL","EFFECT")]
    gene_sub <- gene[as.character(gdsc_sub$CL),]
    outcome$T <- apply(gene_sub,2,get.t,gdsc_sub$EFFECT)
    for (n_feature in n_features){
        subset_id = outcome %>% top_n(n_feature,T) %>% select(Gene)
        subset_id = unname(t(subset_id[,1]))
        gene_sub_final = cbind(gene_sub[,subset_id],gdsc_sub$EFFECT)
        colnames(gene_sub_final)[ncol(gene_sub_final)]='EFFECT' 
        gene_sub_final$EFFECT <- as.factor(gene_sub_final$EFFECT)
        effective_sub = gene_sub_final %>% filter(EFFECT==TRUE)
        ineffective_sub = gene_sub_final %>%  filter(EFFECT==FALSE)
        effective_sub$EFFECT = as.factor(effective_sub$EFFECT)
        ineffective_sub$EFFECT = as.factor(ineffective_sub$EFFECT)
        tune.out = tune ( svm , EFFECT~., data = gene_sub_final , 
                 ranges = list ( cost = c (0.001 , 0.01 , 0.1 , 1 ,10 ,100,1000,10000),gamma=c(0.01,0.1,1,10,100,1000) ) )
        accuracy_train_svm = c()
        accuracy_test_svm = c()
        for (i in 1:8){
            set.seed(i)
            train_effective_id = sample(1:nrow(effective_sub),0.7*nrow(effective_sub))
            train_ineffective_id = sample(1:nrow(ineffective_sub),0.7*nrow(ineffective_sub))
            train_sub = rbind(effective_sub[train_effective_id,],ineffective_sub[train_ineffective_id,])
            test_sub = rbind(effective_sub[-train_effective_id,],ineffective_sub[-train_ineffective_id,])
            mysvm = svm(EFFECT~.,data=train_sub,kernel="radial",cost=tune.out$best.model$cost,gamma=tune.out$best.model$gamma,scale=FALSE)
            # training_pred = predict(mysvm,train_sub)
            # accuracy_train_svm[i] = mean(training_pred==train_sub[['EFFECT']])
            test_pred = predict(mysvm,test_sub)
            accuracy_test_svm[i] = mean(test_pred==test_sub[['EFFECT']])
        }
        resultmat[code,toString(n_feature)] = mean(accuracy_test_svm)
    }
}

In [13]:
saveRDS(resultmat,"data/gene_exp_svm_result.rds")

In [5]:
gdsc_sub <- subset(gdsc, DRUG_ID_lib == '1014')[,c("CL","EFFECT")]
gene_sub <- gene[as.character(gdsc_sub$CL),]

In [6]:
# write a function to calculate the test statistics for a single gene (column)
get.t <- function(dat, labs){
    # split the data into effective and ineffective
    effect <- dat[labs]
    ineffect <- dat[!labs]
    
    # calculate the two sample means
    effect.bar <- mean(effect)
    ineffect.bar <- mean(ineffect)
    
    # calculate the two sample variances
    v.effect <- var(effect)
    v.ineffect <- var(ineffect)
    
    # calculate the sample sizes
    n.effect <- length(effect)
    n.ineffect <- length(ineffect)
    
    # calculate the sd
    s <- sqrt((v.effect/n.effect) + (v.ineffect/n.ineffect))
    
    # calculate the test statistic
    T <- (effect.bar - ineffect.bar)/s
    
    # calculate the degrees of freedom
    # df = ((v.effect/n.effect+v.ineffect/n.ineffect)^2)/(v.effect^2/(n.effect^2 * (n.effect-1))+v.ineffect^2/(n.ineffect^2 * (n.ineffect-1)))
    
    # compare our t value and the threshold, decide whether we should keep it or not
    return(abs(T))
}

In [8]:
outcome <- data.frame(Gene = colnames(gene))

In [8]:
outcome$T <- apply(gene_sub,2,get.t,gdsc_sub$EFFECT)

We try to do SVM with the features that have the most extreme T statistics.

In [90]:
subset_id = outcome %>% top_n(5,T) %>% select(Gene)
subset_id = unname(t(subset_id[,1]))

In [91]:
gene_sub_final = cbind(gene_sub[,subset_id],gdsc_sub$EFFECT)
colnames(gene_sub_final)[6]='EFFECT'

In [92]:
effective_sub = gene_sub_final %>% filter(EFFECT==TRUE)
ineffective_sub = gene_sub_final %>%  filter(EFFECT==FALSE)
effective_sub$EFFECT = as.factor(effective_sub$EFFECT)
ineffective_sub$EFFECT = as.factor(ineffective_sub$EFFECT)

In [93]:
gene_sub_final$EFFECT = as.factor(gene_sub_final$EFFECT)

In [109]:
tune.out = tune ( svm , EFFECT~., data = gene_sub_final , 
                 ranges = list ( cost = c (0.001 , 0.01 , 0.1 , 1 ,10 ,100,1000,10000),gamma=c(0.01,0.1,1,10,100,1000) ) )

In [110]:
tune.out$best.model


Call:
best.tune(method = svm, train.x = EFFECT ~ ., data = gene_sub_final, 
    ranges = list(cost = c(0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000), 
        gamma = c(0.01, 0.1, 0.2, 1, 10, 100, 1000)))


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  100 
      gamma:  0.01 

Number of Support Vectors:  74


In [33]:
mysvm = svm(EFFECT~.,data=gene_sub_final,kernel="linear",cost=tune.out$best.model$cost,scale=FALSE)

In [99]:
accuracy_train_svm = c()
accuracy_test_svm = c()
for (i in 1:8){
    set.seed(i)
    train_effective_id = sample(1:nrow(effective_sub),0.7*nrow(effective_sub))
    train_ineffective_id = sample(1:nrow(ineffective_sub),0.7*nrow(ineffective_sub))
    train_sub = rbind(effective_sub[train_effective_id,],ineffective_sub[train_ineffective_id,])
    test_sub = rbind(effective_sub[-train_effective_id,],ineffective_sub[-train_ineffective_id,])
    mysvm = svm(EFFECT~.,data=train_sub,kernel="radial",cost=tune.out$best.model$cost,gamma=tune.out$best.model$gamma,scale=FALSE)
    training_pred = predict(mysvm,train_sub)
    accuracy_train_svm[i] = mean(training_pred==train_sub[['EFFECT']])
    test_pred = predict(mysvm,test_sub)
    accuracy_test_svm[i] = mean(test_pred==test_sub[['EFFECT']])
}

In [39]:
accuracy_train_svm

[1] 0.8771930 0.8888889 0.8771930 0.8830409 0.8830409 0.8713450 0.8713450
[8] 0.8830409

In [100]:
mean(accuracy_test_svm)

[1] 0.8683333

In [53]:
accuracy_train_svm

[1] 0.8771930 0.8830409 0.8830409 0.8830409 0.8830409 0.8654971 0.8538012
[8] 0.8888889

In [54]:
accuracy_test_svm

[1] 0.8400000 0.8400000 0.8400000 0.8266667 0.8666667 0.8800000 0.8666667
[8] 0.8400000

In [63]:
candidates <- c("1006","1007","1008","1011","1014","1016","1019","1026","1032","1054","1060","1062")

In [68]:
train.accuracies <- rep(0,12)
test.accuracies <- rep(0,12)

names(train.accuracies) <- c("1006","1007","1008","1011","1014","1016","1019","1026","1032","1054","1060","1062")
names(test.accuracies) <- c("1006","1007","1008","1011","1014","1016","1019","1026","1032","1054","1060","1062")

In [65]:
for (j in 1:length(candidates)){
    gdsc_sub <- subset(gdsc, DRUG_ID_lib == candidates[j])[,c("CL","EFFECT")]
    gene_sub <- gene[as.character(gdsc_sub$CL),]
    gene_sub_new = cbind(gene_sub,gdsc_sub['EFFECT'])
    #gene_sub_new$EFFECT = as.factor(gene_sub_new$EFFECT)
    effective_sub = gene_sub_new %>% filter(EFFECT==TRUE)
    ineffective_sub = gene_sub_new %>%  filter(EFFECT==FALSE)
    gene_subset_id = c()
    count = 0
    for (gene_id in gene_label){
        ttestout = t.test(effective_sub[[gene_id]],ineffective_sub[[gene_id]])
        if (ttestout$p.value < 0.05/17737){
            count = count+1
            gene_subset_id[count] = gene_id
        }
    }
    pr.out = prcomp(gene_sub_new[,gene_subset_id])
    gene_sub_reduced = cbind(pr.out$x[,1:3],gene_sub_new[,17738])#pick the first 3 PCs
    colnames(gene_sub_reduced)[4] = 'EFFECT' 
    gene_sub_reduced = as.data.frame(gene_sub_reduced)
    gene_sub_reduced$EFFECT = as.factor(gene_sub_reduced$EFFECT)
    tune.out = tune ( svm , EFFECT~., data = gene_sub_reduced , kernel ="linear",
                 ranges = list ( cost = c (0.001 , 0.01 , 0.05,0.1 , 1 ,5 ,10 ,100) ) )
    effective_sub_reduced = gene_sub_reduced[gene_sub_reduced['EFFECT']==1,]
    ineffective_sub_reduced = gene_sub_reduced[gene_sub_reduced['EFFECT']==0,]
    accuracy_train_svm = c()
    accuracy_test_svm = c()
    for (i in 1:8){
        set.seed(i)
        train_effective_id = sample(1:nrow(effective_sub_reduced),0.7*nrow(effective_sub_reduced))
        train_ineffective_id = sample(1:nrow(ineffective_sub_reduced),0.7*nrow(ineffective_sub_reduced))
        train_sub = rbind(effective_sub_reduced[train_effective_id,],ineffective_sub_reduced[train_ineffective_id,])
        test_sub = rbind(effective_sub_reduced[-train_effective_id,],ineffective_sub_reduced[-train_ineffective_id,])
        mysvm = svm(EFFECT~.,data=train_sub,kernel="linear",cost=tune.out$best.model$cost,scale=FALSE)
        training_pred = predict(mysvm,train_sub)
        accuracy_train_svm[i] = mean(training_pred==train_sub[['EFFECT']])
        test_pred = predict(mysvm,test_sub)
        accuracy_test_svm[i] = mean(test_pred==test_sub[['EFFECT']])
    }
    train.accuracies[candidates[j]] = mean(accuracy_train_svm)
    test.accuracies[candidates[j]] = mean(accuracy_test_svm)
}

In [69]:
train.accuracies

1006      1007      1008      1011      1014      1016      1019      1026 
0.8228610 0.8611919 0.9347826 0.8750000 0.8998538 0.8719136 0.9726124 0.8943662 
     1032      1054      1060      1062 
0.9262153 0.7625740 0.8164286 0.8861111

In [70]:
test.accuracies

1006      1007      1008      1011      1014      1016      1019      1026 
0.7881098 0.8547297 0.9414414 0.8876263 0.8750000 0.8503521 0.9615385 0.8870968 
     1032      1054      1060      1062 
0.9082661 0.7212838 0.7993421 0.8865979

In [48]:
pr.out = prcomp(gene_sub_new[,gene_subset_id])

In [49]:
gene_sub_reduced = cbind(pr.out$x[,1:3],gene_sub_new[,17738])

In [50]:
colnames(gene_sub_reduced)[4] = 'EFFECT' 

In [54]:
gene_sub_reduced = as.data.frame(gene_sub_reduced)
gene_sub_reduced$EFFECT = as.factor(gene_sub_reduced$EFFECT)

In [55]:
tune.out = tune ( svm , EFFECT~., data = gene_sub_reduced , kernel ="linear",
                 ranges = list ( cost = c (0.001 , 0.01 , 0.05,0.1 , 1 ,5 ,10 ,100) ) )

In [56]:
tune.out$best.model$cost

[1] 5

In [57]:
effective_1014 = gene_sub_reduced[gene_sub_reduced['EFFECT']==1,]
ineffective_1014 = gene_sub_reduced[gene_sub_reduced['EFFECT']==0,]

In [58]:
accuracy_train_svm = c()
accuracy_test_svm = c()
for (i in 1:8){
        set.seed(i)
        train_effective_id = sample(1:nrow(effective_1014),0.7*nrow(effective_1014))
        train_ineffective_id = sample(1:nrow(ineffective_1014),0.7*nrow(ineffective_1014))
        train_sub = rbind(effective_1014[train_effective_id,],ineffective_1014[train_ineffective_id,])
        test_sub = rbind(effective_1014[-train_effective_id,],ineffective_1014[-train_ineffective_id,])
        mysvm = svm(EFFECT~.,data=train_sub,kernel="linear",cost=tune.out$best.model$cost,scale=FALSE)
        training_pred = predict(mysvm,train_sub)
        accuracy_train_svm[i] = mean(training_pred==train_sub[['EFFECT']])
        test_pred = predict(mysvm,test_sub)
        accuracy_test_svm[i] = mean(test_pred==test_sub[['EFFECT']])
}

In [62]:
mean(accuracy_train_svm)

[1] 0.8998538

In [61]:
mean(accuracy_test_svm)

[1] 0.875